In [1]:
import numpy as np
import scipy as sc
from scipy.linalg import eigh
import sys
#from scipy.sparse.linalg import eigsh
np.set_printoptions(threshold=sys.maxsize)

### Generación de distintos grafos de conectividad

In [31]:
# 6 vertices 9 aristas

pos = np.array([[1,1,0,0,0,0],[1,0,1,0,0,0],[1,0,0,1,0,0],[1,0,0,0,0,1],[0,1,1,0,0,0],[0,1,0,1,0,0],[0,1,0,0,1,0],[0,0,1,0,1,0],[0,0,1,0,0,1]])




In [23]:
# estrella de david

nodo= [(0, 1),(1, 2),(2, 3),(3, 4),(4, 5),(5, 0),(0, 6),(1, 6),(1, 7),(2, 7),(2, 8),(3, 8),(3, 9),(4, 9),(4, 10),(5, 10),(5, 11),(0, 11)]

def nodoarm(nodo):
    graf = []
    for ele in nodo:
        zeros = np.zeros(12)
        for pos in ele:
            zeros[pos] = 1
        graf.append(list(zeros))    
    return np.array(graf)

pos=nodoarm(nodo)

In [6]:
# 3 vertices 3 aristas
pos = np.array([[1,1,0],[1,0,1],[0,1,1]])

# 6 vertices 6 aristas
pos = np.array([[1,1,0,0,0,0],[0,1,1,0,0,0],[0,0,1,1,0,0],[0,0,0,1,1,0],[0,0,0,0,1,1],[1,0,0,0,0,1]])

#pos = np.array([[1,1,0,0],[0,1,1,0],[0,0,1,1],[0,1,0,1],[1,0,1,0]])

### Generación de matrices de un qubit elementales

In [32]:
Z = np.matrix([[1,0],[0,-1]])
X = np.matrix([[0,1],[1,0]])
Y = np.matrix([[0,-1j],[1j,0]])
I = np.matrix([[1,0],[0,1]])
H = np.matrix([[1,1],[1,-1]])



### Armado del Hamiltoniano del modelo de heisenmerg $\sum_{i,j} X_iX_j+Y_iY_j+Z_iZ_j$

In [33]:
q = pos.shape[1]
pauli =[X,Y,Z]

ham = np.zeros((2**q,2**q))+1j*np.zeros((2**q,2**q))
for pau in pauli:
    for po in pos:
        tira = np.array([[1]])
        for i in range(q):
            tira =np.kron(tira, pau if po[i] ==1 else I)
        ham += tira

ham =ham.real #esto es para eliminar los 0j,  
        

### Obtención de valores y vectores propios del Hamiltoniano

- a autovalores
- v valores propios
- am valor propio mínimo
- vm vectores propios asociados a am

In [34]:
(a,v) =eigh(ham)  #ojo los vectores propios son dados de forma columnas v[0] no es el vector propio asociado a a[0]

v= v.T # ahora si v[0] es el vector propio asociado a a[0]
am = a.min()

vm = v[abs(a-am)<10e-5]

In [39]:
v[2]

array([ 0.00000000e+00,  0.00000000e+00, -6.93889390e-18, -1.17946869e-02,
        6.23354835e-18,  4.90936614e-02, -3.72989745e-02, -1.16226473e-15,
        2.08166817e-17,  1.17792093e-01, -1.46913195e-01, -3.68023702e-02,
       -8.05632609e-02, -1.45043191e-01, -1.54691975e-02, -2.69643993e-01,
        0.00000000e+00, -2.54769546e-01, -1.39325950e-01,  1.60784731e-01,
        4.72347167e-02,  1.60669403e-01,  7.37031840e-02,  8.60316496e-02,
        4.56545143e-01, -1.57415410e-02, -3.69008138e-02,  8.29682654e-02,
       -1.45200206e-01,  1.34863427e-01, -3.42193490e-02, -1.73472348e-18,
        0.00000000e+00,  9.96784786e-02,  3.35332807e-01, -1.45200206e-01,
        2.15338574e-02, -3.69008138e-02, -1.57415410e-02,  1.83612343e-01,
       -3.46860779e-01,  7.37031840e-02,  1.60669403e-01,  8.60898601e-02,
        1.60784731e-01, -6.32485324e-02,  6.31903219e-02,  2.71050543e-19,
       -1.09684363e-01, -1.54691975e-02, -1.45043191e-01, -1.98053749e-01,
       -3.68023702e-02, -

###### Nota curiosa

Si en vez de usar scipy se usa numpy para calcular los eigenvectors ((a,v) np.linalg.eigh(ham)) hay muchos errores numericos que terminan afectando el descubrimiento de los estados propios, es bastante viajero

### Escritura en base computacional de los estados propios

In [36]:

def to_bin(v):
    coef_bin = []
    large = np.log2(len(v))
    for i in range(len(v)):
        if abs(v[i])> 1e-5:
            py_binario = bin(i)[2:]
            binario = '0'*int((large - len(py_binario)))+py_binario
            coef_bin += [(binario,v[i])] 
    return coef_bin
    


In [49]:
for coso in to_bin(v[2]):
    if abs(coso[1]) > 0.05:
        print(coso)

('001001', 0.11779209276581781)
('001010', -0.14691319529801428)
('001100', -0.08056326094085253)
('001101', -0.145043190504976)
('001111', -0.26964399283072504)
('010001', -0.25476954590012485)
('010010', -0.13932595012536797)
('010011', 0.16078473148696268)
('010101', 0.16066940346342995)
('010110', 0.07370318399219153)
('010111', 0.08603164962429724)
('011000', 0.4565451427914147)
('011011', 0.08296826542511276)
('011100', -0.14520020600688197)
('011101', 0.1348634267393591)
('100001', 0.09967847859088727)
('100010', 0.3353328068409763)
('100011', -0.14520020600688188)
('100111', 0.1836123432064278)
('101000', -0.34686077931836534)
('101001', 0.07370318399219188)
('101010', 0.16066940346342912)
('101011', 0.08608986014053047)
('101100', 0.16078473148696137)
('101101', -0.0632485324398982)
('101110', 0.06319032192366492)
('110000', -0.10968436347304898)
('110010', -0.14504319050497688)
('110011', -0.198053748663024)
('110101', -0.05187051118248661)


In [37]:
bin_v = to_bin(vm[0])

for i in bin_v:
    sum = 0
    for j in i[0]:
        sum += int(j)
    if sum != 6:
        print(sum)





3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [14]:
# python code for the above approach
lista = []
# Recursive function that prints
# all strings of N length with equal 1's and 0's
def binaryNum(n, s, ones, zeros):

	# String s contains the output to be printed
	# ones stores the frequency of 1's
	# zeros stores the frequency of 0's
	# Base Condition: When the length of string s
	# becomes N
	if (len(s) == n):
		lista.append(s)
		return

	# If frequency of 1's is less than N/2 then
	# add 1 to the string and increment ones
	if (ones < n / 2):
		binaryNum(n, s + "1", ones + 1, zeros)

	# If frequency of 0's is less than N/2 then
	# add 0 to the string and increment zeros
	if (zeros < n / 2):
		binaryNum(n, s + "0", ones, zeros + 1)

# Driver Code

s = ""
binaryNum(6, s, 0, 0)
print(lista)

# This code is contributed by rakeshsahni


['111000', '110100', '110010', '110001', '101100', '101010', '101001', '100110', '100101', '100011', '011100', '011010', '011001', '010110', '010101', '010011', '001110', '001101', '001011', '000111']


In [21]:
sum = [0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0]

for i in bin_v:

    for j in range(len(i[0])):
        sum[j] += int(i[0][j])



In [16]:
present = []
for i in bin_v:
    present.append(i[0])

c = [x for x in lista if x not in present]
len(c)


0

In [15]:
import scipy as sc
from scipy.linalg import eigh

In [18]:
(a,b) =eigsh(hams)

NameError: name 'eigsh' is not defined

In [19]:
hams= sc.sparse.csr_matrix(ham)